### This notebook is to plot hysteresis patterns for all 7 of my summer storms

Will import all the constituent data, water surface elevation (WSE) and sonde turbidity and fDOM to plot hysteresis

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  

# directories where the CSV files are stored
storm_directory = 'C:/Users/huck4481/Documents/GitHub/La_Jara/Hysteresis/All_Summer_Storms'
sonde_directory = 'C:/Users/huck4481/Documents/GitHub/La_Jara/Hysteresis/All_Summer_Storms/sonde'
WSE_directory = 'C:/Users/huck4481/Documents/GitHub/La_Jara/Hysteresis/All_Summer_Storms/WSE'

# dictionaries to store the data frames
storms = {}
sondes = {}
WSE = {}

# iterate over all files in the directory
# storms:
for filename in os.listdir(storm_directory):
    # check if the file is a CSV file
    if filename.endswith('.csv'):
        file_path = os.path.join(storm_directory, filename) # construct the full file path
        df = pd.read_csv(file_path)                         # read the CSV file into a data frame
        df = df.dropna(subset=['Date/Time'])                # drop rows where 'Date/Time' is NaN     
        key = filename[:-4]                                 # remove the '.csv' from the filename to use as the dictionary key
        storms[key] = df                                    # store the data frame in the dictionary

# sondes:
for filename in os.listdir(sonde_directory):
    # check if the file is a CSV file
    if filename.endswith('.csv'):
        file_path = os.path.join(sonde_directory, filename)
        df = pd.read_csv(file_path)
        key = filename[:-4]
        sondes[key] = df

# WSE:
for filename in os.listdir(WSE_directory):
    # check if the file is a CSV file
    if filename.endswith('.csv'):
        file_path = os.path.join(WSE_directory, filename)
        df = pd.read_csv(file_path)
        key = filename[:-4]
        WSE[key] = df

In [2]:
storms.keys()

dict_keys(['st1_down', 'st1_up', 'st2_down', 'st3_down', 'st4_down', 'st4_up', 'st5_down', 'st5_up', 'st6_up', 'st7_down', 'st7_up', 'WSE_2021', 'WSE_2022', 'WSE_2023'])

In [5]:
storms['st1_up']

,Date/Time,WSE,Q,SS (uL/L),SRP (mg/L),TP (mg/L),DOC (mg/L),POC (mg/L),N (mg/L),PP (mg/L),LAB ID,fDOM,Turbidity
0,7/23/2021 14:54,NaN,NaN,92.060,0.042,0.094,4.522,11.105057,NaN,0.15,NaN,329.294,8.208
1,7/23/2021 15:02,NaN,NaN,95.690,0.045,0.120,4.436,29.712121,NaN,0.28,NaN,339.230,7.452
2,7/23/2021 16:13,NaN,NaN,403.515,0.090,0.362,12.442,41.279351,NaN,1.66,NaN,388.274,122.774
3,7/23/2021 16:58,NaN,NaN,124.040,0.083,0.208,12.977,9.524065,NaN,0.38,NaN,630.966,42.612
4,7/23/2021 17:58,NaN,NaN,65.300,0.067,0.132,13.108,6.538644,NaN,0.13,NaN,742.718,21.154
5,7/23/2021 20:13,NaN,NaN,42.870,0.048,0.086,12.073,5.707480,NaN,0.06,NaN,779.800,10.370
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
